# Program - print out profiles TaiESM

**Content:**
Print out profiles in TaiESM, given time step, lat, and lon.

This is useful for diagnosing the tendencies. 

**Author**: Yi-Hsuan Chen (yihsuan@umich.edu)

In [103]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import io, os, sys, types

import yhc_module as yhc

xr.set_options(keep_attrs=True)  # keep attributes after xarray operation

## define functions

### read_var

In [104]:
#--- TaiESM hindcast simulation results

def read_var(varname, 
             time_step=0, lat=0, lon=120, # lat & lon are in degrees
             date = "2001-07-03",
             datapath = "/lfs/home/yihsuanc/test/d1-taiesm-hindcast-data/y1-hindcast_2001July-taiesm1.F_2000_TAI.f09_f09/run/",
             filename_head = "y1-hindcast_2001July-taiesm1.F_2000_TAI.f09_f09.icdate_20010701.cam", 
             ):

    func_name = "read_var"
    
    #--- varnames
    varnames_Ttend = ["TTEND_TOT","DTCORE","PTTEND","ZMDT","EVAPTZM","FZSNTZM","EVSNTZM","ZMMTT","CMFDT","DPDLFT","SHDLFT", "MACPDT","MPDT","QRL","QRS","DTV","TTGWORO"]
    varnames_Qtend = ["PTEQ","ZMDQ","EVAPQZM","CMFDQ","MACPDQ","MPDQ","VD01", "PTECLDLIQ","ZMDLIQ","CMFDLIQ","DPDLFLIQ","SHDLFLIQ","MACPDLIQ","MPDLIQ","VDCLDLIQ","PTECLDICE","ZMDICE","CMFDICE","DPDLFICE","SHDLFICE","MACPDICE","MPDICE","VDCLDICE"]
    
    if varname in varnames_Ttend:
        filename = filename_head+".h2_Ttend_3h."+date+"-00000.nc"

    elif varname in varnames_Qtend:
        filename = filename_head+".h3_Qtend_3h."+date+"-00000.nc"

    else:
        filename = "sdflkjsdlkfjsldf"

    #--- read file
    infile = datapath+"/"+filename
    ds_taiesm = xr.open_dataset(infile)
    
    #--- read variable
    da_taiesm = ds_taiesm[varname][time_step, :, :, :].sel(lat=lat, lon=lon, method="nearest")

    #--- Modify units if needed
    cp_air = 1005 # specific heat of air, units: J/kg/K

    if (varname == "MACPDT" or varname == "MPDT"):
        da_taiesm = da_taiesm / cp_air
        da_taiesm.attrs['units'] = "K/s"
    
    return da_taiesm

#da_taiesm = read_var("PTEQ", )
#da_taiesm #.values
#da_taiesm.PTEQ.sel(lat=50, lon=100, method="nearest")

#da = ds_taiesm[varname](, :, :, :)..sel(lat=lat, lon=lon, method="nearest")

#-----------
# do_test
#-----------

#do_test=True
do_test=False

if (do_test):
    tt=3
    lat0 = 0
    lon0 = 100
    varname = "ZMDT"

    var = read_var(varname, time_step=tt, lat=lat0, lon=lon0)

    var

### create_code

In [105]:
#--- T tendencies
#varnames = ["TTEND_TOT", "DTCORE", "PTTEND", "ZMDT", "ZMMTT", "EVAPTZM", "CMFDT", "DPDLFT", "SHDLFT", "MACPDT", "MPDT","QRL","QRS", "DTV", "TTGWORO"]

#--- Q tendencies
#varnames = ["PTEQ", "ZMDQ", "EVAPQZM", "CMFDQ", "MACPDQ", "MPDQ", "VD01"]

#--- cldliq tendencies
#varnames = ["PTECLDLIQ", "ZMDLIQ", "CMFDLIQ", "DPDLFLIQ", "SHDLFLIQ", "MACPDLIQ", "MPDLIQ", "VDCLDLIQ"]

#--- cldice tendencies
#varnames = ["PTECLDICE","ZMDICE", "CMFDICE", "DPDLFICE", "SHDLFLIQ", "MACPDICE", "MPDICE", "VDCLDICE"]
 
varnames= [
        "TTEND_TOT","DTCORE","PTTEND","ZMDT","EVAPTZM","FZSNTZM","EVSNTZM","ZMMTT","CMFDT","DPDLFT","SHDLFT", "MACPDT","MPDT","QRL","QRS","DTV","TTGWORO",
        "PTEQ","ZMDQ","EVAPQZM","CMFDQ","MACPDQ","MPDQ","VD01", "PTECLDLIQ","ZMDLIQ","CMFDLIQ","DPDLFLIQ","SHDLFLIQ","MACPDLIQ","MPDLIQ","VDCLDLIQ","PTECLDICE","ZMDICE","CMFDICE","DPDLFICE","SHDLFICE","MACPDICE","MPDICE","VDCLDICE"
]


#-----------
# do_test
#-----------

#do_test=True
do_test=False

if (do_test):
    for var in varnames:
        text = f"{var} = read_var(\"{var}\", time_step=tt, lat=lat0, lon=lon0)"
        print(text)

    ss = " + ".join(varnames)
    print(ss)
    

### print_max_min

In [106]:
def print_max_min(rrr):
    print("")    
    print("Max rrr: " + str(np.nanmax(np.abs(rrr.values))))
    print("Min rrr: " + str(np.nanmin(np.abs(rrr.values))))

## Read all variables

In [107]:
#--- set time step, lat0, and lon0
tt=0
lat0 = 60
lon0 = 60

#--- read variables
TTEND_TOT = read_var("TTEND_TOT", time_step=tt, lat=lat0, lon=lon0)
DTCORE = read_var("DTCORE", time_step=tt, lat=lat0, lon=lon0)
PTTEND = read_var("PTTEND", time_step=tt, lat=lat0, lon=lon0)
ZMDT = read_var("ZMDT", time_step=tt, lat=lat0, lon=lon0)
EVAPTZM = read_var("EVAPTZM", time_step=tt, lat=lat0, lon=lon0)
FZSNTZM = read_var("FZSNTZM", time_step=tt, lat=lat0, lon=lon0)
EVSNTZM = read_var("EVSNTZM", time_step=tt, lat=lat0, lon=lon0)
ZMMTT = read_var("ZMMTT", time_step=tt, lat=lat0, lon=lon0)
CMFDT = read_var("CMFDT", time_step=tt, lat=lat0, lon=lon0)
DPDLFT = read_var("DPDLFT", time_step=tt, lat=lat0, lon=lon0)
SHDLFT = read_var("SHDLFT", time_step=tt, lat=lat0, lon=lon0)
MACPDT = read_var("MACPDT", time_step=tt, lat=lat0, lon=lon0)
MPDT = read_var("MPDT", time_step=tt, lat=lat0, lon=lon0)
QRL = read_var("QRL", time_step=tt, lat=lat0, lon=lon0)
QRS = read_var("QRS", time_step=tt, lat=lat0, lon=lon0)
DTV = read_var("DTV", time_step=tt, lat=lat0, lon=lon0)
TTGWORO = read_var("TTGWORO", time_step=tt, lat=lat0, lon=lon0)
PTEQ = read_var("PTEQ", time_step=tt, lat=lat0, lon=lon0)
ZMDQ = read_var("ZMDQ", time_step=tt, lat=lat0, lon=lon0)
EVAPQZM = read_var("EVAPQZM", time_step=tt, lat=lat0, lon=lon0)
CMFDQ = read_var("CMFDQ", time_step=tt, lat=lat0, lon=lon0)
MACPDQ = read_var("MACPDQ", time_step=tt, lat=lat0, lon=lon0)
MPDQ = read_var("MPDQ", time_step=tt, lat=lat0, lon=lon0)
VD01 = read_var("VD01", time_step=tt, lat=lat0, lon=lon0)
PTECLDLIQ = read_var("PTECLDLIQ", time_step=tt, lat=lat0, lon=lon0)
ZMDLIQ = read_var("ZMDLIQ", time_step=tt, lat=lat0, lon=lon0)
CMFDLIQ = read_var("CMFDLIQ", time_step=tt, lat=lat0, lon=lon0)
DPDLFLIQ = read_var("DPDLFLIQ", time_step=tt, lat=lat0, lon=lon0)
SHDLFLIQ = read_var("SHDLFLIQ", time_step=tt, lat=lat0, lon=lon0)
MACPDLIQ = read_var("MACPDLIQ", time_step=tt, lat=lat0, lon=lon0)
MPDLIQ = read_var("MPDLIQ", time_step=tt, lat=lat0, lon=lon0)
VDCLDLIQ = read_var("VDCLDLIQ", time_step=tt, lat=lat0, lon=lon0)
PTECLDICE = read_var("PTECLDICE", time_step=tt, lat=lat0, lon=lon0)
ZMDICE = read_var("ZMDICE", time_step=tt, lat=lat0, lon=lon0)
CMFDICE = read_var("CMFDICE", time_step=tt, lat=lat0, lon=lon0)
DPDLFICE = read_var("DPDLFICE", time_step=tt, lat=lat0, lon=lon0)
SHDLFICE = read_var("SHDLFICE", time_step=tt, lat=lat0, lon=lon0)
MACPDICE = read_var("MACPDICE", time_step=tt, lat=lat0, lon=lon0)
MPDICE = read_var("MPDICE", time_step=tt, lat=lat0, lon=lon0)
VDCLDICE = read_var("VDCLDICE", time_step=tt, lat=lat0, lon=lon0)

sum_T_phys = (ZMDT + ZMMTT + EVAPTZM + CMFDT + DPDLFT + SHDLFT + MACPDT + MPDT + QRL + QRS + DTV + TTGWORO).rename("sum_T_phys")
sum_Q_phys = (ZMDQ + EVAPQZM + CMFDQ + MACPDQ + MPDQ + VD01).rename("sum_Q_phys")
sum_ql_phys = (ZMDLIQ + CMFDLIQ + DPDLFLIQ + SHDLFLIQ + MACPDLIQ + MPDLIQ + VDCLDLIQ).rename("sum_ql_phys")
sum_qi_phys = (ZMDICE + CMFDICE + DPDLFICE + SHDLFLIQ + MACPDICE + MPDICE + VDCLDICE).rename("sum_qi_phys")

VDCLDICE

<xarray.DataArray 'VDCLDICE' (lev: 30)>
[30 values with dtype=float32]
Coordinates:
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
    time     object 2001-07-03 00:00:00
    lat      float64 59.84
    lon      float64 60.0
Attributes:
    mdims:         1
    units:         kg/kg/s
    long_name:     Vertical diffusion of CLDICE
    cell_methods:  time: mean

## Print out values

### T tendencies - total

In [108]:
rrr = TTEND_TOT-DTCORE-PTTEND
yhc.print_1d_arrays(TTEND_TOT, DTCORE, PTTEND, rrr)

print_max_min(rrr)

Index	lev	TTEND_TOT	DTCORE	PTTEND	Data4
1	3.64346569404006	-8.255171997006983e-05	-1.3545123692892957e-05	-6.900659354869276e-05	0.0
2	7.594819646328688	-5.272239286568947e-05	-1.2953743862453848e-05	-3.976864900323562e-05	0.0
3	14.356632251292467	-2.02837836695835e-05	5.129052169650095e-06	-2.5412835384486243e-05	0.0
4	24.612220004200935	8.039904059842229e-06	2.5698385798023082e-05	-1.7658481738180853e-05	0.0
5	38.26829977333546	-2.170894731534645e-06	1.0791384738695342e-05	-1.2962279470229987e-05	0.0
6	54.59547974169254	-1.8544144040788524e-05	-9.486786439083517e-06	-9.057357601705007e-06	0.0
7	72.01245054602623	-1.7173875676235184e-05	-1.0668047252693214e-05	-6.505829333036672e-06	9.094947017729282e-13
8	87.82123029232025	-1.144136240327498e-05	-5.365844117477536e-06	-6.075518740544794e-06	4.547473508864641e-13
9	103.31712663173676	-2.216209213656839e-05	-1.737850470817648e-05	-4.7835860641498584e-06	-1.3642420526593924e-12
10	121.54724076390266	-1.6152578609762713e-05	-1.2417238394

### T tendencies - physics

In [109]:
rrr = ((PTTEND-sum_T_phys)/PTTEND).rename('relative diff')
yhc.print_1d_arrays(PTTEND, sum_T_phys, PTTEND-sum_T_phys, rrr)

#print("Max rrr: "+ str (max(abs(rrr.values))) )
#print("Min rrr: "+ str (min(abs(rrr.values))) )
print_max_min(rrr)

Index	lev	PTTEND	sum_T_phys	Data3	relative diff
1	3.64346569404006	-6.900659354869276e-05	-6.900659354869276e-05	0.0	-0.0
2	7.594819646328688	-3.976864900323562e-05	-3.976865264121443e-05	3.637978807091713e-12	-9.147856161462187e-08
3	14.356632251292467	-2.5412835384486243e-05	-2.5412835384486243e-05	0.0	-0.0
4	24.612220004200935	-1.7658481738180853e-05	-1.7658481738180853e-05	0.0	-0.0
5	38.26829977333546	-1.2962279470229987e-05	-1.2962280379724689e-05	9.094947017729282e-13	-7.016471670340252e-08
6	54.59547974169254	-9.057357601705007e-06	-9.057358511199709e-06	9.094947017729282e-13	-1.004150149697125e-07
7	72.01245054602623	-6.505829333036672e-06	-6.505829333036672e-06	0.0	-0.0
8	87.82123029232025	-6.075518740544794e-06	-6.075518740544794e-06	0.0	-0.0
9	103.31712663173676	-4.7835860641498584e-06	-4.783586518897209e-06	4.547473508864641e-13	-9.506410947324184e-08
10	121.54724076390266	-3.735340214916505e-06	-3.735340214916505e-06	0.0	-0.0
11	142.99403876066208	-5.288542070047697e-06	-5

### Q tendencies - physics

In [110]:
rrr = ((PTEQ-sum_Q_phys)/PTEQ).rename('relative diff')
yhc.print_1d_arrays(PTEQ, sum_Q_phys, PTEQ-sum_Q_phys, rrr)
print_max_min(rrr)


Index	lev	PTEQ	sum_Q_phys	Data3	relative diff
1	3.64346569404006	7.842897644240471e-26	0.0	7.842897644240471e-26	1.0
2	7.594819646328688	7.842897644240471e-26	0.0	7.842897644240471e-26	1.0
3	14.356632251292467	7.842897644240471e-26	0.0	7.842897644240471e-26	1.0
4	24.612220004200935	0.0	0.0	0.0	nan
5	38.26829977333546	1.1764346774509498e-25	0.0	1.1764346774509498e-25	1.0
6	54.59547974169254	1.1764346774509498e-25	0.0	1.1764346774509498e-25	1.0
7	72.01245054602623	3.921448822120236e-26	0.0	3.921448822120236e-26	1.0
8	87.82123029232025	7.842897644240471e-26	0.0	7.842897644240471e-26	1.0
9	103.31712663173676	1.1764346774509498e-25	0.0	1.1764346774509498e-25	1.0
10	121.54724076390266	7.842897644240471e-26	0.0	7.842897644240471e-26	1.0
11	142.99403876066208	0.0	2.692645303401468e-28	-2.692645303401468e-28	-inf
12	168.22507977485657	3.5018539013832155e-23	3.4823980671055687e-23	1.9455834277646858e-25	0.005555866751819849
13	197.9080867022276	2.1543419837519233e-20	2.1543578165058354e-20	-1.58

### ql tendencies - physics

In [111]:
rrr = ((PTECLDLIQ-sum_ql_phys)/PTECLDLIQ).rename('relative diff')
yhc.print_1d_arrays(PTECLDLIQ, sum_ql_phys, PTECLDLIQ-sum_ql_phys, rrr)
print('=====================')
#yhc.print_1d_arrays(PTECLDLIQ, sum_ql_phys, MACPDLIQ, MPDLIQ, DPDLFLIQ, rrr)
print('=====================')
#yhc.print_1d_arrays(sum_ql_phys, ZMDLIQ, CMFDLIQ, DPDLFLIQ, SHDLFLIQ, MACPDLIQ, MPDLIQ, VDCLDLIQ, rrr)

print_max_min(rrr)

Index	lev	PTECLDLIQ	sum_ql_phys	Data3	relative diff
1	3.64346569404006	0.0	0.0	0.0	nan
2	7.594819646328688	0.0	0.0	0.0	nan
3	14.356632251292467	0.0	0.0	0.0	nan
4	24.612220004200935	0.0	0.0	0.0	nan
5	38.26829977333546	0.0	0.0	0.0	nan
6	54.59547974169254	0.0	0.0	0.0	nan
7	72.01245054602623	0.0	0.0	0.0	nan
8	87.82123029232025	0.0	0.0	0.0	nan
9	103.31712663173676	0.0	0.0	0.0	nan
10	121.54724076390266	0.0	0.0	0.0	nan
11	142.99403876066208	0.0	0.0	0.0	nan
12	168.22507977485657	0.0	0.0	0.0	nan
13	197.9080867022276	0.0	0.0	0.0	nan
14	232.82861895859241	0.0	0.0	0.0	nan
15	273.9108167588711	0.0	0.0	0.0	nan
16	322.2419023513794	0.0	0.0	0.0	nan
17	379.10090386867523	-1.8168886662437493e-35	0.0	-1.8168886662437493e-35	1.0
18	445.992574095726	-3.3597740493863306e-37	0.0	-3.3597740493863306e-37	1.0
19	524.6871747076511	-0.0	0.0	-0.0	nan
20	609.7786948084831	-0.0	0.0	-0.0	nan
21	691.3894303143024	-0.0	0.0	-0.0	nan
22	763.404481112957	-0.0	0.0	-0.0	nan
23	820.8583686500788	-2.0456938920609957e-10	-2.04

### qi tendencies - physics

In [112]:
rrr = ((PTECLDICE-sum_qi_phys)/PTECLDICE).rename('relative diff')
yhc.print_1d_arrays(PTECLDICE, sum_qi_phys, PTECLDICE-sum_qi_phys, rrr)
print('=====================')
yhc.print_1d_arrays(PTECLDICE, sum_qi_phys, MACPDICE, MPDICE, DPDLFICE, SHDLFICE, VDCLDICE, rrr)

print_max_min(rrr)

Index	lev	PTECLDICE	sum_qi_phys	Data3	relative diff
1	3.64346569404006	0.0	0.0	0.0	nan
2	7.594819646328688	0.0	0.0	0.0	nan
3	14.356632251292467	0.0	0.0	0.0	nan
4	24.612220004200935	0.0	0.0	0.0	nan
5	38.26829977333546	0.0	0.0	0.0	nan
6	54.59547974169254	0.0	0.0	0.0	nan
7	72.01245054602623	0.0	0.0	0.0	nan
8	87.82123029232025	0.0	0.0	0.0	nan
9	103.31712663173676	0.0	0.0	0.0	nan
10	121.54724076390266	0.0	0.0	0.0	nan
11	142.99403876066208	0.0	0.0	0.0	nan
12	168.22507977485657	-3.481259898591516e-23	-3.481259898591516e-23	0.0	-0.0
13	197.9080867022276	-2.1543662175589317e-20	-2.1543662175589317e-20	0.0	-0.0
14	232.82861895859241	-8.804573771030075e-20	-8.804573771030075e-20	0.0	-0.0
15	273.9108167588711	-1.3598870222517418e-22	-1.3598870222517418e-22	0.0	-0.0
16	322.2419023513794	-2.175754422790814e-23	-2.175754422790814e-23	0.0	-0.0
17	379.10090386867523	-5.379467831098133e-25	-5.379467831098133e-25	0.0	-0.0
18	445.992574095726	-3.7972135525324283e-28	-3.7972135525324283e-28	0.0	-0.0
19	524